In [3]:
from math import pow
from pulp import (
    LpProblem, LpMinimize, LpVariable, LpBinary, LpContinuous,
    lpSum, PULP_CBC_CMD, LpStatus, value
)

# ----- datos ---- #
P = (1, 2)                              # 1=Pequeña, 2=Grande
C = (1, 2, 3, 4, 5, 6)                  # ciudades
A = (1, 2, 3)                           # años
R = (1, 2, 3, 4, 5, 6)                  # regiones
T = (1, 2, 3)                           # modos transporte AT1..AT3
CIUDAD = {1:"Antofagasta",2:"Valparaiso",3:"Santiago",4:"Rancagua",5:"Concepcion",6:"Puerto Montt"}

# Tabla 1 (demanda base y crecimiento)
D0 = {1: 951_776, 2: 967_364, 3: 512_051, 4: 386_248, 5: 946_174, 6: 303_445}
porcentaje = {1: 0.16, 2: 0.22, 3: 0.26, 4: 0.15, 5: 0.39, 6: 0.30}

D = {(r,a): D0[r]*pow(1.0+porcentaje[r], a) for r in R for a in A}

CAP = {1: 4_636_446, 2: 14_966_773}
PEQ, GRA = 1, 2

CF = {
  (PEQ,1): 18_236_639, (PEQ,2):  8_838_286, (PEQ,3):  6_840_758,
  (PEQ,4): 13_378_246, (PEQ,5): 26_394_217, (PEQ,6):  3_678_737,
  (GRA,1): 60_788_796, (GRA,2): 32_734_393, (GRA,3): 32_575_039,
  (GRA,4): 53_512_984, (GRA,5): 65_985_543, (GRA,6): 26_276_695,
}

CP = {
  (PEQ,1): 28.20, (PEQ,2): 41.68, (PEQ,3): 18.57, (PEQ,4): 17.68, (PEQ,5): 50.11, (PEQ,6): 43.55,
  (GRA,1): 28.20, (GRA,2): 41.68, (GRA,3): 18.57, (GRA,4): 17.68, (GRA,5): 50.11, (GRA,6): 43.55,
}

CA = {
  (PEQ,1):  86_626_147, (PEQ,2): 115_721_215, (PEQ,3): 172_235_977,
  (PEQ,4):           0, (PEQ,5):  57_494_934, (PEQ,6): 175_561_277,
  (GRA,1): 201_456_157, (GRA,2): 199_519_337, (GRA,3): 291_925_385,
  (GRA,4): 299_031_830, (GRA,5): 179_671_671, (GRA,6): 337_617_842,
}

AT1 = {
  1:[1.06, 2.80,10.29, 4.87, 6.41,10.35],
  2:[3.49, 6.19, 3.39, 6.77, 3.07, 6.61],
  3:[6.38, 5.88, 5.63, 1.01, 3.15, 5.67],
  4:[3.44, 1.48, 2.79, 2.80, 5.30, 1.29],
  5:[5.94, 7.33, 1.80, 9.48, 2.82, 8.25],
  6:[2.57, 9.63, 4.84, 6.64, 6.48, 8.54],
}
AT2 = {
  1:[10.03, 4.09, 4.55, 7.84, 5.33,10.63],
  2:[10.52, 1.82, 3.91, 8.20, 5.88, 2.33],
  3:[ 1.90, 8.89, 6.55, 9.71, 7.03,10.23],
  4:[ 2.06,10.17, 2.12, 6.11, 3.79, 6.19],
  5:[ 2.54, 6.95, 8.57,10.50, 4.85, 5.31],
  6:[ 7.92,10.32, 1.41, 4.94, 2.74, 8.08],
}
AT3 = {
  1:[ 9.86, 4.30, 8.10, 9.63, 7.40, 6.47],
  2:[ 1.58, 2.71, 3.08, 5.91, 7.99, 5.11],
  3:[ 9.13,10.03, 6.77, 5.70, 3.62, 8.58],
  4:[ 8.95, 7.37, 10.29, 3.34, 2.21, 4.58],
  5:[ 9.62, 3.78, 5.19, 2.61, 3.19, 1.78],
  6:[10.32, 8.88, 10.87,10.38, 5.83, 1.54],
}

CT = {}
for c in C:
    for r in R:
        CT[(c,r,1)] = AT1[c][r-1]
        CT[(c,r,2)] = AT2[c][r-1]
        CT[(c,r,3)] = AT3[c][r-1]

# ---------------------------
# Modelo 
# ---------------------------
def build_model(CF, CA, CP, CAP, CT, D, O_is_binary=True, rancagua_city=4, flows_integer=False):
    problem = LpProblem("Funnys_company", LpMinimize)

    X = LpVariable.dicts("X", [(p,c,a) for p in P for c in C for a in A], 0, 1, LpBinary)
    # ← cat correcto según bandera
    O_cat = LpBinary if O_is_binary else LpContinuous
    O = LpVariable.dicts("O", [(p,c,a) for p in P for c in C for a in A], 0, 1, O_cat)

    flow_cat = LpContinuous if not flows_integer else "Integer"
    Amount_var = LpVariable.dicts(
        "A", [(p,c,a,r,t) for p in P for c in C for a in A for r in R for t in T],
        lowBound=0, cat=flow_cat
    )

    # Objetivo (corrige CT[(c,r,t)]).
    # Nota: si el costo fijo CF es por apertura única, reemplaza CF*X por CF*O.
    problem += (
        lpSum(CF[(p,c)] * X[(p,c,a)] for p in P for c in C for a in A)  # (o usa CF*O si es one-off)
      + lpSum(CA[(p,c)] * O[(p,c,a)] for p in P for c in C for a in A)
      + lpSum((CP[(p,c)] + CT[(c,r,t)]) * Amount_var[(p,c,a,r,t)]
              for p in P for c in C for a in A for r in R for t in T)
    )

    # Aperturas sin año 0 (c != Rancagua)
    for p in P:
        for c in C:
            if c == rancagua_city:
                continue
            # a=1
            problem += O[(p,c,1)] == X[(p,c,1)]
            # a=2,3
            for a in (2,3):
                problem += O[(p,c,a)] >= X[(p,c,a)] - X[(p,c,a-1)]
                problem += O[(p,c,a)] <= X[(p,c,a)]
                problem += O[(p,c,a)] <= 1 - X[(p,c,a-1)]

    # Rancagua fija: pequeña siempre, grande prohibida; sin aperturas
    for a in A:
        problem += X[(1, rancagua_city, a)] == 1
        problem += X[(2, rancagua_city, a)] == 0
        problem += O[(1, rancagua_city, a)] == 0
        problem += O[(2, rancagua_city, a)] == 0

    # Demanda por región y año
    for r in R:
        for a in A:
            problem += lpSum(Amount_var[(p,c,a,r,t)] for p in P for c in C for t in T) >= D[(r,a)]

    # Capacidad por planta (p,c,a)
    for p in P:
        for c in C:
            for a in A:
                problem += lpSum(Amount_var[(p,c,a,r,t)] for r in R for t in T) <= CAP[p] * X[(p,c,a)]

    # Una alternativa por ciudad-año
    for c in C:
        for a in A:
            problem += lpSum(X[(p,c,a)] for p in P) <= 1

    return problem, X, O, Amount_var

if __name__ == "__main__":
    # O binaria, flujos continuos por defecto:
    problem, X, O, Amount_var = build_model(CF, CA, CP, CAP, CT, D, O_is_binary=True, flows_integer=False)

    solver = PULP_CBC_CMD(msg=True)
    problem.solve(solver)

    print("Status:", LpStatus[problem.status])
    print("Costo total:", value(problem.objective))

    for a in A:
        print(f"\nAÑO {a}")
        for c in C:
            opened = [p for p in P if X[(p,c,a)].value() and X[(p,c,a)].value() > 0.5]
            for p in opened:
                print(f"  Ciudad {c} ({CIUDAD[c]}): Planta {'Peq' if p==1 else 'Gra'} abierta")
        for c in C:
            for p in P:
                if O[(p,c,a)].value() and O[(p,c,a)].value() > 1e-6:
                    print(f"    * Apertura en a={a}: p={p}, c={c} ({CIUDAD[c]})")

    print("\nArcos con flujo > 0:")
    for key, var in Amount_var.items():
        if var.value() and var.value() > 1e-3:
            p,c,a,r,t = key
            print(f"  A[p={p},c={c},a={a},r={r},t={t}] = {var.value():.2f}")


Status: Optimal
Costo total: 619887153.8034601

AÑO 1
  Ciudad 3 (Santiago): Planta Peq abierta
  Ciudad 4 (Rancagua): Planta Peq abierta
    * Apertura en a=1: p=1, c=3 (Santiago)

AÑO 2
  Ciudad 3 (Santiago): Planta Peq abierta
  Ciudad 4 (Rancagua): Planta Peq abierta

AÑO 3
  Ciudad 3 (Santiago): Planta Peq abierta
  Ciudad 4 (Rancagua): Planta Peq abierta

Arcos con flujo > 0:
  A[p=1,c=3,a=1,r=1,t=2] = 2642.86
  A[p=1,c=3,a=1,r=4,t=1] = 444185.20
  A[p=1,c=3,a=2,r=1,t=2] = 1237945.40
  A[p=1,c=3,a=2,r=4,t=1] = 510812.98
  A[p=1,c=3,a=3,r=1,t=2] = 1485623.40
  A[p=1,c=3,a=3,r=4,t=1] = 587434.93
  A[p=1,c=3,a=3,r=5,t=1] = 1352166.10
  A[p=1,c=4,a=1,r=1,t=2] = 1101417.30
  A[p=1,c=4,a=1,r=2,t=1] = 1180184.10
  A[p=1,c=4,a=1,r=3,t=2] = 645184.26
  A[p=1,c=4,a=1,r=5,t=3] = 1315181.90
  A[p=1,c=4,a=1,r=6,t=1] = 394478.50
  A[p=1,c=4,a=2,r=1,t=2] = 42764.42
  A[p=1,c=4,a=2,r=2,t=1] = 1439824.60
  A[p=1,c=4,a=2,r=3,t=2] = 812932.17
  A[p=1,c=4,a=2,r=5,t=3] = 1828102.80
  A[p=1,c=4,a=2,r=